In [11]:
import csv
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit( (x['User-ID'] for x in csv.DictReader(open("data/aux_data_headers_2.csv"),delimiter=";")),
             (x['ISBN'] for x in csv.DictReader(open("data/aux_data_headers_2.csv"),delimiter=";"))) 

print('ready')

ready


In [6]:
#num_users, num_items = dataset.interactions_shape()
#print('Num users: {}, num_items {}.'.format(num_users, num_items))


#print(dataset._user_id_mapping)
#print(dataset._item_id_mapping)
#print(dataset._user_feature_mapping)
#print(dataset._item_feature_mapping)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11, '13': 12, '14': 13, '15': 14, '16': 15, '17': 16, '18': 17, '19': 18, '20': 19, '21': 20, '22': 21, '23': 22, '24': 23, '25': 24, '26': 25, '27': 26, '28': 27, '29': 28, '30': 29, '31': 30, '32': 31, '33': 32, '34': 33, '35': 34, '36': 35, '37': 36, '38': 37, '39': 38, '40': 39, '41': 40, '42': 41, '43': 42, '44': 43, '45': 44, '46': 45, '47': 46, '48': 47, '49': 48, '50': 49, '51': 50, '52': 51, '53': 52, '54': 53, '55': 54, '56': 55, '57': 56, '58': 57, '59': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '70': 69, '71': 70, '72': 71, '73': 72, '74': 73, '75': 74, '76': 75, '77': 76, '78': 77, '79': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '88': 87, '89': 88, '90': 89, '91': 90, '92': 91, '93': 92, '94': 93, '95': 94, '96': 95, '97': 96, '98': 97, '99': 98, '100': 99, '101': 100, '102'

In [3]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

print('ready')

Num users: 7770, num_items 135.
ready


In [4]:
(interactions, weights) = dataset.build_interactions(( (x['User-ID'], x['ISBN'], float(x['Book-Rating']))
          for x in csv.DictReader(open("data/aux_data_headers_2.csv"),delimiter=";" ) ))

print('ready')

ready


In [9]:
print(type(interactions))

print('ready')

<class 'scipy.sparse.coo.coo_matrix'>
ready


In [10]:
print(repr(interactions))
print(interactions.todense()[251,0])
print(interactions.todense()[8,20])
print(interactions.todense()[70,15])
print(interactions.todense()[9,5])

<7770x135 sparse matrix of type '<class 'numpy.float32'>'
	with 1048575 stored elements in COOrdinate format>
1.0
1.0
1.0
1.0


In [20]:
from lightfm.lightfm import LightFM

model = LightFM(loss='bpr')
model.fit(interactions)

In [26]:
from lightfm.evaluation import precision_at_k
from lightfm.datasets import fetch_movielens
import numpy as np

def sample_recommendation(model,data, user_ids):
    n_user, n_items = data.shape                                   #define sizes
    for user_id in user_ids:
        known_positives = data.todense()[user_id]                  # get the already known items
        scores = model.predict(user_id, np.arange(n_items))        # make prediction for new items
        top_items = data.todense()[np.argsort(-scores)]            # sort best recommendations
        
        print("user %s" % user_id)
        print(" known positives:")
        for x in known_positives[:5]:
            print(" %s" % x)

        print(" recommended:")
        for x in top_items[:5]:
            print(" %s" % x)
            
        print(" scores:")
        print(" %s"% np.argsort(-scores))

test_precision = precision_at_k(model, interactions, k=10).mean()
print('presicion: ',test_precision)

sample_recommendation(model,interactions,[4, 25, 450])

presicion:  1.0
user 4
 known positives:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]]
 recommended:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0